# Data Preparation and Merging Pipeline

## Step 1: Load Libraries and Data

In [1]:

import pandas as pd
import numpy as np

data_2006_2023_path = "processed_data/data_2006_2023.csv"
elecBalance_path = "processed_data/elecBalance.csv"
energyPrice_path = "processed_data/energyPrice.csv"
GDP_path =  "processed_data/GDP.csv"
nao_path =  "processed_data/nao.csv"
populationNL_path =  "processed_data/populationNL.csv"
renewableEnergy_path = "processed_data/renwableEnergy.csv"
weather_path = "processed_data/weather.csv"
yearlyFinalConsPerSource_path =  "processed_data/yearlyfinalConsPerSource.csv"

In [2]:
loadConsumption = pd.read_csv(data_2006_2023_path)
weather = pd.read_csv(weather_path)
elecBalance = pd.read_csv(elecBalance_path)
gdp = pd.read_csv(GDP_path)
populationNL = pd.read_csv(populationNL_path)
renewableEnergy = pd.read_csv(renewableEnergy_path)
yearlyFinalConsPerSource = pd.read_csv(yearlyFinalConsPerSource_path)
energyPrice = pd.read_csv(energyPrice_path)
nao = pd.read_csv(nao_path)

## Step 2: Preprocess Data

### 2.1 Transform Hourly Data to Daily Data

In [3]:
#loadConsumption.head()

# Ensure 'datetime' is in datetime format
loadConsumption['datetime'] = pd.to_datetime(loadConsumption['datetime'], errors='coerce')

# Group by the date and sum up the load consumption
daily_loadConsumption = loadConsumption.groupby(loadConsumption['datetime'].dt.date).agg({'loadConsumption': 'sum'}).reset_index()

daily_loadConsumption.rename(columns={'datetime': 'Date', 'loadConsumption': 'DailyLoadConsumption'}, inplace=True)



### 2.2 Standardize Yearly Data

In [4]:

# Ensure Year column is an integer
datasets_yearly = [yearlyFinalConsPerSource, elecBalance, energyPrice, gdp, populationNL, renewableEnergy, nao]
for dataset in datasets_yearly:
    dataset['Year'] = dataset['Year'].astype(int)


## Step 3: Merge Datasets

### 3.1 Merge Daily Datasets

In [5]:
weather['Date'] = pd.to_datetime(weather['Date'])
daily_loadConsumption['Date'] = pd.to_datetime(daily_loadConsumption['Date'])

In [6]:
weather_loadConsumption = pd.merge(weather, daily_loadConsumption, on = 'Date')

### 3.2 Merge Yearly Datasets

In [7]:
# Group the dataset by 'PriceComponents' and create separate DataFrames for each component
price_components_dfs = {
    component: df for component, df in energyPrice.groupby("PriceComponents")
}

# Display the keys (i.e., the names of each component)
print("Separate DataFrames created for each PriceComponent:")
print(list(price_components_dfs.keys()))

totalPrice_energy = price_components_dfs["TotalPrice"]
#totalPrice_energy.head()

Separate DataFrames created for each PriceComponent:
['DeliveryPrice', 'NetworkPrice', 'TotalPrice']


In [8]:
# Filter datasets to include only rows where Year is between 2009 and 2022
renewableEnergy = renewableEnergy[(renewableEnergy['Year'] >= 2009) & (renewableEnergy['Year'] <= 2022)]
populationNL = populationNL[(populationNL['Year'] >= 2009) & (populationNL['Year'] <= 2022)]
yearlyFinalConsPerSource = yearlyFinalConsPerSource[(yearlyFinalConsPerSource['Year'] >= 2009) & (yearlyFinalConsPerSource['Year'] <= 2022)]
elecBalance = elecBalance[(elecBalance['Year'] >= 2009) & (elecBalance['Year'] <= 2022)]

# Merge yearly datasets on Year
yearly_data = renewableEnergy.merge(populationNL, on='Year', how='outer')
yearly_data = yearly_data.merge(yearlyFinalConsPerSource, on='Year', how='outer')
yearly_data = yearly_data.merge(elecBalance, on='Year', how='outer')


In [9]:
yearly_data.head()

,Year,GeothermalHeat,AmbientAirHeat,TotalGeothermal&GroundEnergy,TotalBiomass,TotalRes(excl.Stat.Transfer),TotalRes(incl.Stat.Transfer),Totaal windenergie,TotalSolarEnergy,Total Including Non-Renewable,...,Exports (PJ),Imports (PJ),Industry (PJ),"Oil refineries, transformation (PJ)",Other final consumption (PJ),Production (PJ),Residential (PJ),Total energy supply (PJ),Total final consumption (PJ),Transport (PJ)
0,2009,1841.0,351.0,1983.0,73747.0,93660.0,93660.0,16131.0,1088.0,2195534.0,...,-38.019598,55.627198,131.312882,0,25.498801,0,82.339200,17.607600,376.760879,6.040799
1,2010,2183.0,536.0,2501.0,71605.0,92411.0,92411.0,16210.0,1196.0,2359420.0,...,-46.108801,56.098799,140.767371,0,25.286400,0,82.778399,9.990002,387.511372,6.321599
2,2011,2538.0,737.0,2854.0,76881.0,99256.0,99256.0,17010.0,1416.0,2192915.0,...,-41.511599,74.232002,140.438448,0,25.415999,0,82.954802,32.720399,387.956449,6.267602
3,2012,2852.0,961.0,3347.0,78470.0,102676.0,102676.0,17780.0,1757.0,2203277.0,...,-54.165105,115.760603,125.853634,0,28.801403,0,84.107985,61.595498,375.176334,6.346799
4,2013,3147.0,1230.0,4140.0,75825.0,103463.0,103463.0,19324.0,2582.0,2205198.0,...,-54.054000,119.707202,125.246112,0,30.124051,0,84.099452,65.653198,376.386709,6.299999


### 3.3 Upsample Yearly Data to Daily Data

In [10]:

# Convert Yearly data to daily frequency
yearly_data['Date'] = pd.to_datetime(yearly_data['Year'], format='%Y')  # Create a date column from Year
daily_from_yearly = yearly_data.set_index('Date').resample('D').ffill().reset_index()


### 3.4 Merge, and upsample the res of the data

In [13]:
# Step 1: Standardize dates for all datasets
totalPrice_energy['Year'] = totalPrice_energy['Periods'].str[:4]
totalPrice_energy['Half'] = totalPrice_energy['Periods'].str.extract(r'(\d)e')
half_to_month = {'1': '-01-15', '2': '-07-15'}
totalPrice_energy['Date'] = pd.to_datetime(totalPrice_energy['Year'] + totalPrice_energy['Half'].map(half_to_month))
totalPrice_energy = totalPrice_energy[['Date', 'NG_Household_LessThan_569m3_EuroPerM3','NG_Household_569to5687m3_EuroPerM3',
 'NG_Household_MoreThan_5687m3_EuroPerM3','NG_NonHousehold_LessThan_28433m3_EuroPerM3','NG_NonHousehold_28433to284333m3_EuroPerM3',
 'NG_NonHousehold_284333to2843332m3_EuroPerM3','NG_NonHousehold_2843332to28433324m3_EuroPerM3','NG_NonHousehold_MoreThan_28433324m3_EuroPerM3',
 'Electricity_Household_LessThan_1MWh_EuroPerKWh','Electricity_Household_1to2.5MWh_EuroPerKWh','Electricity_Household_2.5to5MWh_EuroPerKWh',
 'Electricity_Household_5to15MWh_EuroPerKWh','Electricity_Household_MoreThan_15MWh_EuroPerKWh','Electricity_NonHousehold_LessThan_20MWh_EuroPerKWh',
 'Electricity_NonHousehold_20to500MWh_EuroPerKWh','Electricity_NonHousehold_500to2000MWh_EuroPerKWh','Electricity_NonHousehold_2000to20000MWh_EuroPerKWh',
 'Electricity_NonHousehold_20000to70000MWh_EuroPerKWh','Electricity_NonHousehold_70000to150000MWh_EuroPerKWh','Electricity_NonHousehold_MoreThan_150000MWh_EuroPerKWh',
 'AveragePrice_Electricity_Household','AveragePrice_Electricity_NonHousehold','AveragePrice_NaturalGas_Household','AveragePrice_NaturalGas_NonHousehold']]  

gdp['Date'] = pd.to_datetime(gdp['Period'])  # Use existing 'Period' column as dates
gdp = gdp[['Date', 'GDP']]  # Select relevant columns

nao['Date'] = pd.to_datetime(nao['Year'].astype(str) + '-' + nao['Month'].astype(str) + '-01')  # Combine Year/Month
nao = nao[['Date', 'NAO']]  # Select relevant columns

# Step 2: Resample NAO and GDP to daily frequency
# NAO: Resample to daily, forward-fill values within each month
nao_daily = nao.set_index('Date').resample('D').ffill().reset_index()

# GDP: Resample to daily, forward-fill values within each quarter
gdp_daily = gdp.set_index('Date').resample('D').ffill().reset_index()

# Step 3: Filter all datasets for years 2009–2022
start_date = '2009-01-15'
end_date = '2022-12-31'

totalPrice_energy = totalPrice_energy[(totalPrice_energy['Date'] >= start_date) & (totalPrice_energy['Date'] <= end_date)]
gdp_daily = gdp_daily[(gdp_daily['Date'] >= start_date) & (gdp_daily['Date'] <= end_date)]
nao_daily = nao_daily[(nao_daily['Date'] >= start_date) & (nao_daily['Date'] <= end_date)]

# Step 4: Merge datasets
merged_data = pd.merge(totalPrice_energy, gdp_daily, on='Date', how='outer')
merged_data = pd.merge(merged_data, nao_daily, on='Date', how='outer')

# Step 5: Upsample final merged dataset to daily frequency (in case of any remaining gaps)
merged_data = merged_data.set_index('Date').resample('D').mean()

# Step 6: Interpolate missing values
merged_data = merged_data.interpolate(method='linear')

# Save and display the final dataset
merged_data.head()


,NG_Household_LessThan_569m3_EuroPerM3,NG_Household_569to5687m3_EuroPerM3,NG_Household_MoreThan_5687m3_EuroPerM3,NG_NonHousehold_LessThan_28433m3_EuroPerM3,NG_NonHousehold_28433to284333m3_EuroPerM3,NG_NonHousehold_284333to2843332m3_EuroPerM3,NG_NonHousehold_2843332to28433324m3_EuroPerM3,NG_NonHousehold_MoreThan_28433324m3_EuroPerM3,Electricity_Household_LessThan_1MWh_EuroPerKWh,Electricity_Household_1to2.5MWh_EuroPerKWh,...,Electricity_NonHousehold_2000to20000MWh_EuroPerKWh,Electricity_NonHousehold_20000to70000MWh_EuroPerKWh,Electricity_NonHousehold_70000to150000MWh_EuroPerKWh,Electricity_NonHousehold_MoreThan_150000MWh_EuroPerKWh,AveragePrice_Electricity_Household,AveragePrice_Electricity_NonHousehold,AveragePrice_NaturalGas_Household,AveragePrice_NaturalGas_NonHousehold,GDP,NAO
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-15,1.057000,0.791000,0.722000,0.752000,0.662000,0.459000,0.374000,0.276000,-0.103000,0.144000,...,0.118000,0.108000,0.104000,0.080000,0.142400,0.134429,0.856667,0.504600,158583.0,-0.01
2009-01-16,1.056856,0.790265,0.721171,0.751282,0.661724,0.458713,0.373757,0.275801,-0.103077,0.143956,...,0.117989,0.107950,0.103945,0.079956,0.142302,0.134369,0.856098,0.504255,158583.0,-0.01
2009-01-17,1.056713,0.789530,0.720343,0.750564,0.661448,0.458425,0.373514,0.275602,-0.103155,0.143912,...,0.117978,0.107901,0.103890,0.079912,0.142203,0.134310,0.855529,0.503910,158583.0,-0.01
2009-01-18,1.056569,0.788796,0.719514,0.749845,0.661171,0.458138,0.373271,0.275403,-0.103232,0.143867,...,0.117967,0.107851,0.103834,0.079867,0.142105,0.134251,0.854959,0.503566,158583.0,-0.01
2009-01-19,1.056425,0.788061,0.718685,0.749127,0.660895,0.457851,0.373028,0.275204,-0.103309,0.143823,...,0.117956,0.107801,0.103779,0.079823,0.142007,0.134192,0.854390,0.503221,158583.0,-0.01


### 3.4 Combine Daily and Upsampled Yearly Data

In [14]:

# Merge daily data with upsampled yearly data
fist_daily_merge = pd.merge(weather_loadConsumption, daily_from_yearly, on='Date', how='inner')
final_data_daily = pd.merge(fist_daily_merge, merged_data, on='Date', how='inner')


## Step 4: Validation and Optimization

In [15]:
final_data_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4735 entries, 0 to 4734
Data columns (total 81 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   Date                                                    4735 non-null   datetime64[ns]
 1   DailyPrecipitation                                      4735 non-null   float64       
 2   MaxHourlyPrecipitation                                  4735 non-null   float64       
 3   HDMaxPrecipitation                                      4735 non-null   int64         
 4   DailyMeanTemperature                                    4735 non-null   float64       
 5   HourlyMinTemperature                                    4735 non-null   int64         
 6   HDMinTemperature                                        4735 non-null   int64         
 7   HourlyMaxTemperature                                    4735

In [17]:
# Save the final dataset
final_data_daily.to_csv("final_data_daily.csv", index=False)
